In [1]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm import tqdm_notebook

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [4]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [5]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.001) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.999))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [6]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [7]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [8]:
#generate_new_params(2000)

In [9]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [10]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [11]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [12]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [13]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [14]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [15]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [16]:
probabilities, d_target, g_res = sample_from_csv(100)
A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [17]:
def agents_list_only_IC_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.001
    alpha = [[proba/(1 - (risk_aversion[i])) for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [100000 for proba in probabilities]

    j_max = [10 for i in range(community_size)]

    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [18]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [45]:
def gurobi_experiment(df_param, index, err_track, agents, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):

    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)
    print(risk_aversion[2])
    if solution_type == 'only_IC':
        agents = agents_list_only_IC_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()
        
        
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [20]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [21]:
connection_matrix_2 = text_to_adj_matrix('../matrices/matrix_2.txt')

In [22]:
def experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='only_IC', verbosity = 0):

    #if solution_type == 'only_IC':
    #    agents = agents_list_only_IC_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    total_results= {}
    err_track = []
    for index in tqdm_notebook(df_param.index.levels[0]):
        vars, objective, model = gurobi_experiment(df_param, index, err_track, agents, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm_notebook(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [46]:
res_only_IC, df_only_IC, model_only_IC = experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res,
            df_param, solution_type='only_IC')

/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_51019/266194919.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm_notebook(df_param.index.levels[0]):


  0%|          | 0/2000 [00:00<?, ?it/s]

0.9900079883477656
0.7370153457537029
0.2438447027082295
0.3350110663277154
0.6335116926328919
0.4262338402940124
0.6805229360393305
0.0608377993715304
0.986220611237964
0.3598407483029202
0.4340121234692936
0.6707964087224664
0.9850165594528916
0.0410449569938202
0.3925645529688791


KeyboardInterrupt: 

In [24]:
df_only_IC

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,J_24_91,J_24_92,J_24_93,J_24_94,J_24_95,J_24_96,J_24_97,J_24_98,J_24_99,objective
0,2.647996,2.661296,2.674396,2.687318,2.700073,2.712705,2.725210,2.737668,2.750116,2.762599,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
1,3.163335,3.189869,3.216715,3.243857,3.271339,3.299160,3.327301,3.355754,3.384506,3.413538,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
2,3.166436,3.184173,3.202093,3.220201,3.238442,3.256812,3.275359,3.294059,3.312908,3.331908,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
3,3.305012,3.330124,3.355522,3.381218,3.407201,3.433448,3.459950,3.486686,3.513619,3.540647,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
4,2.736738,2.747287,2.757856,2.768273,2.778765,2.789333,2.799983,2.810720,2.821550,2.832487,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.064570,3.080614,3.096872,3.113350,3.130048,3.146970,3.164074,3.181384,3.198973,3.216861,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
1996,2.949755,2.961700,2.973747,2.985768,2.997798,3.009916,3.023338,3.037785,3.052465,3.067129,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
1997,3.070835,3.089829,3.108961,3.128227,3.147626,3.167158,3.186745,3.206436,3.226240,3.246137,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0
1998,2.761198,2.773262,2.784742,2.795713,2.806262,2.816460,2.826460,2.836392,2.846346,2.856241,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0


In [29]:
df_only_IC.to_csv(f'../data/only_IC_results_{len(df_only_IC)}.csv')

In [25]:
def agent_utility_from_df_row(df_row, agents):
    res = []
    for agent in agents:
        eta_idx = f'eta_{agent.id}'
        agent_obj = df_row[eta_idx]

        for proba in agent.probabilities_ind:
            u_idx = f'u_{agent.id}_{proba}'
            w_idx = f'W_{agent.id}_{proba}'
            j_idx = f'J_{agent.id}_{proba}'

            agent_obj += (agent.alpha[proba] * df_row[j_idx]
                        + agent.gamma[proba] * df_row[w_idx]
                        + agent.probabilities[proba] / (1 - agent.risk_aversion) * df_row[u_idx])

        res.append(agent_obj)

    return res

In [26]:
def IC_utility_from_df_row(df_row, agents):
    res = 0

    for agent in agents:
        for proba in agent.probabilities_ind:
            j_idx = f'J_{agent.id}_{proba}'

            res += - agent.alpha[proba] * df_row[j_idx] + agent.probabilities[proba] * df_row[j_idx]

    return res

In [27]:
agents = agents_list_only_IC_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)

In [28]:
agent_objectives = []
for _, row in df_only_IC.iterrows():
    agent_objectives.append(agent_utility_from_df_row(row, agents))

In [30]:
dict_df_param = df_param.to_dict()
for i in range(community_size):
    for trial_idx, trial in enumerate(agent_objectives):
        dict_df_param[f'{i}'].update({(trial_idx, 'objective') : trial[i]})

df_param = pd.DataFrame(dict_df_param).sort_index(level=0)
df_param.to_csv(f'../data/only_IC_params_{2000}_with_obj.csv')

In [31]:
IC_objective = []
for _, row in df_only_IC.iterrows():
    IC_objective.append(IC_utility_from_df_row(row, agents))

In [32]:
IC_objective = pd.Series(IC_objective, name='IC objective only_IC')
IC_objective.to_csv('../data/IC_objective_only_IC_2000.csv')

In [33]:
IC_objective

0      -1521.194046
1      -1521.194046
2      -1521.194046
3      -1521.194046
4      -1521.194046
           ...     
1995   -1521.194046
1996   -1521.194046
1997   -1521.194046
1998   -1521.194046
1999   -1521.194046
Name: IC objective only_IC, Length: 2000, dtype: float64

In [44]:
df_param.loc[3]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
A_tilde,0.159778,0.496107,0.730656,0.453526,0.318792,0.920719,0.858758,0.842005,0.813446,0.136520,...,0.719766,0.556374,0.926457,0.051792,0.506975,0.084709,0.872965,0.969384,0.777111,0.762512
B_tilde,0.766138,0.489117,0.103509,0.471848,0.578028,0.227691,0.233844,0.535174,0.061477,0.173811,...,0.973027,0.023463,0.544667,0.131003,0.057309,0.528059,0.520104,0.817861,0.324032,0.469493
a,0.763572,0.042176,0.838613,0.713404,0.756326,0.603921,0.763589,0.445480,0.662402,0.888914,...,0.799197,0.730056,0.622557,0.318488,0.101620,0.108528,0.749964,0.126604,0.143414,0.854627
b,0.980755,0.128922,0.304952,0.527549,0.324982,0.243576,0.226001,0.295683,0.381414,0.725718,...,0.531870,0.475552,0.420119,0.712076,0.901138,0.056356,0.301511,0.921272,0.261415,0.741392
d,0.315922,0.633019,0.013726,0.421763,0.761246,0.664305,0.033646,0.440269,0.040916,0.334305,...,0.313312,0.349471,0.838716,0.510505,0.217066,0.489792,0.051286,0.364423,0.242561,0.035338
objective,33.899185,48.377449,9959.701457,167.830829,65.209456,65.551159,136.517904,98.311736,26.968655,83.994270,...,25.294166,129.113011,602.791335,34.604874,1888.740465,44.003073,202.023579,133.680723,39.138998,535.407227
risk_aversion,0.740251,0.332970,0.335011,0.356864,0.459099,0.628247,0.263039,0.185999,0.052336,0.669031,...,0.458564,0.378369,0.661728,0.888302,0.421365,0.491891,0.331401,0.743673,0.706058,0.906955


In [89]:
columns = []
for i in range(100):
    for agent in agents:
        if agents[2].connections[agent.id]:
            columns.append(f'q_2_{agent.id}_{i}')

df_only_IC.loc[3][columns].describe()

count    300.000000
mean       0.164412
std        0.750677
min       -0.651855
25%       -0.535373
50%       -0.254307
75%        1.022883
max        1.320111
Name: 3, dtype: float64

In [100]:
columns = []
for i in range(100):
    columns.append(f'J_2_{i}')

df_only_IC.loc[3][columns].describe()

count    100.0
mean      10.0
std        0.0
min       10.0
25%       10.0
50%       10.0
75%       10.0
max       10.0
Name: 3, dtype: float64

In [94]:
df_only_IC.loc[3]['eta_2']

-74.93600830265468

In [97]:
agents[2].G_res

array([0.     , 0.04556, 0.09112, 0.13668, 0.18224, 0.2278 , 0.27336,
       0.31892, 0.36448, 0.41004, 0.4556 , 0.50116, 0.54672, 0.59228,
       0.63784, 0.6834 , 0.72896, 0.77452, 0.82008, 0.86564, 0.9112 ,
       0.95676, 1.00232, 1.04788, 1.09344, 1.139  , 1.18456, 1.23012,
       1.27568, 1.32124, 1.3668 , 1.41236, 1.45792, 1.50348, 1.54904,
       1.5946 , 1.64016, 1.68572, 1.73128, 1.77684, 1.8224 , 1.86796,
       1.91352, 1.95908, 2.00464, 2.0502 , 2.09576, 2.14132, 2.18688,
       2.23244, 2.278  , 2.32356, 2.36912, 2.41468, 2.46024, 2.5058 ,
       2.55136, 2.59692, 2.64248, 2.68804, 2.7336 , 2.77916, 2.82472,
       2.87028, 2.91584, 2.9614 , 3.00696, 3.05252, 3.09808, 3.14364,
       3.1892 , 3.23476, 3.28032, 3.32588, 3.37144, 3.417  , 3.46256,
       3.50812, 3.55368, 3.59924, 3.6448 , 3.69036, 3.73592, 3.78148,
       3.82704, 3.8726 , 3.91816, 3.96372, 4.00928, 4.05484, 4.1004 ,
       4.14596, 4.19152, 4.23708, 4.28264, 4.3282 , 4.37376, 4.41932,
       4.46488, 4.51

In [98]:
agent_utility_from_df_row(df_only_IC.loc[3], agents)

[33.89918471743481,
 48.377449205567935,
 9959.701456953291,
 167.83082881541162,
 65.20945572383725,
 65.55115918772272,
 136.5179044046457,
 98.31173627623669,
 26.968654844254303,
 83.99427027309139,
 64.69517810012769,
 113.60695691537659,
 39.848237915017016,
 117.82267404425669,
 52.68590922379084,
 25.294165790540628,
 129.1130110407329,
 602.7913348387737,
 34.60487363860348,
 1888.740465482708,
 44.003073372091926,
 202.0235787578592,
 133.68072330306563,
 39.13899845770755,
 535.4072271180694]

In [102]:
sum(agent_objectives[0])

6783.456160985313

In [106]:
agent = agents[2]

eta_idx = 'eta_2'
agent_obj = df_only_IC.loc[3][eta_idx]
print(agent_obj)

for proba in agent.probabilities_ind:
    u_idx = f'u_2_{proba}'
    w_idx = f'W_2_{proba}'
    j_idx = f'J_2_{proba}'

    agent_obj += (agent.alpha[proba] * df_only_IC.loc[3][j_idx]
                + agent.gamma[proba] * df_only_IC.loc[3][w_idx]
                + agent.probabilities[proba] / (1 - agent.risk_aversion) * df_only_IC.loc[3][u_idx])
    
    print(agent_obj)

-74.93600830265468
1660.3100558093677
2815.4879134536895
4227.997410651598
4980.179171606401
5436.747632561159
5743.163394929691
6065.988988382362
6272.956697389583
6385.771338304866
6480.532203687979
6573.250716725199
6645.0145902252025
6732.856566566478
6814.5035790281145
6905.102572546012
6991.4998186338535
7062.616050193038
7140.984029878154
7216.901856090784
7282.67595260433
7344.96603794185
7416.3844807781825
7479.422216220928
7540.117492656037
7590.071248400344
7642.070580348688
7698.812754668633
7745.720411255522
7799.250658721312
7847.73215199986
7895.560910971392
7936.561977478426
7984.5027439602
8035.444756914083
8087.145587673689
8137.926589341279
8192.813159069714
8252.350413955015
8322.928109368282
8401.459520958872
8477.40297118525
8566.272570052663
8647.763582061803
8743.52106610764
8848.274460796274
8944.942137843755
9052.53583229529
9132.271810353026
9195.465424574682
9241.590992214953
9275.615913316324
9295.361811111228
9314.824331716603
9384.97073618172
9437.5627910

In [107]:
df_only_IC.loc[3][j_idx]

10.0

In [114]:
agents[6].alpha

[0.24239539489922352,
 0.1619806404177307,
 0.19881684696958027,
 0.10627410037559239,
 0.06475114432942282,
 0.04361965976731273,
 0.046127481865150664,
 0.02968274679736075,
 0.01623917587944246,
 0.013690241943935063,
 0.013443570917918143,
 0.010442406768046513,
 0.01282689335287606,
 0.011963544761817059,
 0.013320235404909756,
 0.012744669677537136,
 0.010524630443385438,
 0.011634650060461358,
 0.011305755359105515,
 0.00982572920300443,
 0.009332387150970737,
 0.010730189631732895,
 0.009496834501648586,
 0.009167939800292886,
 0.00756457813118327,
 0.007893472832539114,
 0.008633485910589584,
 0.007153459754488501,
 0.008181255696225352,
 0.007071236079149575,
 0.00698901240381065,
 0.006002328299743262,
 0.007030124241480112,
 0.007482354455844345,
 0.007605689968852732,
 0.007482354455844345,
 0.008099032020886427,
 0.00879793326126758,
 0.010442406768046513,
 0.011634650060461358,
 0.011264643521436052,
 0.013196899891901369,
 0.012127992112495054,
 0.014265807671307683,
 0